# Modelling for Global Health - Data science in Python
## Day 4: Neural networks in Python

Carrying on from yesterday, we will look to use the feature-rich, gene expression data to practise classification using neural networks. Can we acurrately classify the samples to the experimental conditions based on their gene expression?

In [ ]:
# Packages for importing, cleaning and looking at the data
import pandas as pd
import numpy as np
from pathlib import Path
from natsort import index_natsorted, order_by_index
import random
from collections import Counter
import keras
from keras.models import Sequential, model_from_json
from keras.layers import *
import seaborn as sns

In [ ]:
# Load the dataset
expr_new_info = pd.read_csv('Common_info_data.txt', index_col=0)

expr_naive = pd.read_csv('Common_naive_data.txt', index_col=0)
expr_ifn = pd.read_csv('Common_ifn_data.txt', index_col=0)
expr_lps2 = pd.read_csv('Common_lps2_data.txt', index_col=0)
expr_lps24 = pd.read_csv('Common_lps24_data.txt', index_col=0)

In [ ]:
#  Discover the dataset
for expr in [expr_naive, expr_ifn, expr_lps2, expr_lps24]:
    print(expr.shape)
    print(expr.head(5))

In [ ]:
# For normalizing the data
def initial_normalize(df):
    result = df.copy()
    feature_max = {}
    feature_min = {}
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        feature_max[feature_name] = max_value + 0.1
        min_value = df[feature_name].min() - 0.1
        feature_min[feature_name] = min_value
        if max_value == 0:
            result[feature_name]= df[feature_name]
        else:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result, feature_max, feature_min

In [ ]:
# Let's combine the dataset and normalize it
samps_in_common = list(set(expr_naive.columns.values) & set(expr_ifn.columns.values) & set(expr_lps2.columns.values) & set(expr_lps24.columns.values))

expr_all_treat = pd.concat([expr_naive[samps_in_common].T,expr_ifn[samps_in_common].T,expr_lps24[samps_in_common].T,expr_lps2[samps_in_common].T], keys=['Naive', 'IFN', 'LPS24', 'LPS2'])
expr_all_treat_norm, feature_max, feature_min = initial_normalize(expr_all_treat)
classes = ['Naive', 'IFN', 'LPS24', 'LPS2']

In [ ]:
# Set the training and testing sets
rows = random.sample(range(len(expr_all_treat_norm.index)), int(.75*len(expr_all_treat_norm.index)))
rows.sort()

training = expr_all_treat_norm.values[rows,]
training_labels = np.array(expr_all_treat_norm.index.get_level_values(0)[rows])
testing = np.delete(expr_all_treat_norm.values,rows,axis=0)
testing_labels = np.array(np.delete(expr_all_treat_norm.index.get_level_values(0), [rows], axis=0))

training_samples = Counter(training_labels)
print('For training set we have the following samples:')
for key in training_samples:
    print(key, training_samples[key])

testing_samples = Counter(testing_labels)
print('For testing set we have the following samples:')
for key in testing_samples:
    print(key, testing_samples[key])

In [ ]:
# Now preprocess the data
scaled_training = training
scaled_testing = testing

scaled_training_labels = np.zeros((len(training),len(classes)))
for i,tr in enumerate(training_labels):
    scaled_training_labels[i,classes.index(tr)] = 1
scaled_testing_labels = np.zeros((len(testing),len(classes)))
for i,tr in enumerate(testing_labels):
    scaled_testing_labels[i,classes.index(tr)] = 1

In [ ]:
# Create model
x_train = scaled_training
y_train = scaled_training_labels

x_test = scaled_testing
y_test = scaled_testing_labels

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(50, input_dim=17867, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(4, activation="softmax"))

In [ ]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print a summary of the model
model.summary()

In [ ]:
# Train the model
model.fit(
    x_train,
    y_train,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)

In [ ]:
test_error_rate = model.evaluate(x_test, y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

In [ ]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure_dense.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights_dense.h5")

# # Load the json file that contains the model's structure
# f = Path("data/ks2a_model_structure_dense.json")
# model_structure = f.read_text()
#
# # Recreate the Keras model object from the json data
# model = model_from_json(model_structure)
#
# # Re-load the model's trained weights
# model.load_weights("data/ks2a_model_weights_dense.h5")

In [ ]:
# Predict non-matched samples
# Remove samples already used in making the model
expr_naive_p = expr_naive.drop(samps_in_common,axis=1).T
expr_ifn_p = expr_ifn.drop(samps_in_common, axis=1).T
expr_lps24_p = expr_lps24.drop(samps_in_common, axis=1).T
expr_lps2_p = expr_lps2.drop(samps_in_common, axis=1).T

In [ ]:
# For normalizing the data the same way as before
def subsidary_normalize(df, feature_max, feature_min):
    result = df.copy()
    for feature_name in df.columns:
        max_value = feature_max[feature_name]
        min_value = feature_min[feature_name]
        if max_value == 0:
            result[feature_name]= df[feature_name]
        else:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
expr_all_eval = pd.concat([expr_naive_p, expr_ifn_p, expr_lps24_p, expr_lps2_p], keys=['Naive', 'IFN', 'LPS24', 'LPS2'])
expr_all_eval_norm = subsidary_normalize(expr_all_eval, feature_max, feature_min)
evaluation = expr_all_eval_norm.values
evaluation_labels = np.array(expr_all_eval_norm.index.get_level_values(0))

evaluation_samples = Counter(evaluation_labels)
print('For evaluation set we have the following samples:')
for key in evaluation_samples:
    print(key, evaluation_samples[key])

In [ ]:
scaled_evaluation = evaluation

scaled_evaluation_labels = np.zeros((len(evaluation),len(classes)))
for i,tr in enumerate(evaluation_labels):
    scaled_evaluation_labels[i,classes.index(tr)] = 1

In [ ]:
scaled_evaluation_labels

In [ ]:
# Make a prediction with the neural network
X = scaled_evaluation
prediction = model.predict(X)
df_pred = pd.DataFrame(data=prediction, index=expr_all_eval.index, columns=['Naive', 'IFN', 'LPS24', 'LPS2'])

In [ ]:
df_pred.style.background_gradient(cmap='viridis')

In [ ]:
#%% Predict the new rna-seq samples
classes = ['IFNG', 'LPS2', 'LPS24', 'LPS6', 'UT']
count_data_common = pd.read_csv('Common_count_data.txt', index_col=0)
rna_data = count_data_common.T
rna_data['sample'] = [x.split("_")[0] for x in rna_data.index]
rna_data['treatment'] = [x.split("_")[1] for x in rna_data.index]
rna_data = rna_data.set_index(['treatment', 'sample'])
rna_data.sort_index(inplace=True)

In [ ]:
rna_eval, _, _ = initial_normalize(rna_data)
rna_evaluation = rna_eval.values
rna_evaluation_labels = np.array(rna_eval.index.get_level_values(0))

rna_evaluation_samples = Counter(rna_evaluation_labels)
print('For RNA evaluation set we have the following samples:')
for key in rna_evaluation_samples:
    print(key, rna_evaluation_samples[key])

In [ ]:
rna_scaled_evaluation = rna_evaluation

rna_scaled_evaluation_labels = np.zeros((len(rna_evaluation),len(classes)))
for i,tr in enumerate(rna_evaluation_labels):
    rna_scaled_evaluation_labels[i,classes.index(tr)] = 1

In [ ]:
# Make a prediction with the neural network
X = rna_scaled_evaluation
rna_prediction = model.predict(X)
df_rna_pred = pd.DataFrame(data=rna_prediction, index=rna_data.index, columns=['Naive', 'IFN', 'LPS24', 'LPS2'])

In [ ]:
df_rna_pred.style.background_gradient(cmap='viridis')

## Different neural network models
### Dense layers, no loss
So far we have used one of the simplest Neural Networks with dense layers and no loss, though it possible that we are over fitting. 

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(50, input_dim=17867, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(4, activation="softmax"))

### Dense layers, with loss
One way to avoid overfitting, is to randomly add loss to the network, so not all the data makes it from layer to layer. 

In [ ]:
# New model with loss
model = Sequential()
model.add(Dense(50, input_dim=17867, activation='relu', name='layer_1'))
model.add(Dropout(0.20))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dropout(0.20))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dropout(0.20))
model.add(Dense(4, activation="softmax"))

# # Save neural network structure
# model_structure = model.to_json()
# f = Path("model_structure_loss.json")
# f.write_text(model_structure)

# # Save neural network's trained weights
# model.save_weights("model_weights_loss.h5")

In [ ]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print a summary of the model
model.summary()

In [ ]:
# Train the model
model.fit(
    x_train,
    y_train,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)

In [ ]:
test_error_rate = model.evaluate(x_test, y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

In [ ]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure_loss.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights_loss.h5")

# # Load the json file that contains the model's structure
# f = Path("data/ks2b_model_structure_loss.json")
# model_structure = f.read_text()
#
# # Recreate the Keras model object from the json data
# model = model_from_json(model_structure)
#
# # Re-load the model's trained weights
# model.load_weights("ks2b_model_weights_loss.h5")

In [ ]:
# Make a prediction with the neural network - non-matched samples
X = scaled_evaluation
prediction = model.predict(X)
df_pred = pd.DataFrame(data=prediction, index=expr_all_eval.index, columns=['Naive', 'IFN', 'LPS24', 'LPS2'])
df_pred.style.background_gradient(cmap='viridis')

In [ ]:
# Make a prediction with the neural network
X = rna_scaled_evaluation
rna_prediction = model.predict(X)
df_rna_pred = pd.DataFrame(data=rna_prediction, index=rna_data.index, columns=['Naive', 'IFN', 'LPS24', 'LPS2'])
df_rna_pred.style.background_gradient(cmap='viridis')

### Convolutional layer with loss
Loss just on dense layers has too much of a detrimental effect on our results. How about if we segment our data, but keep the loss between layers in?
Note the input of this data into a convolution layer needs an extra dimension, which then needs to be pooled and flattened before beingn put into a dense layer. 

In [ ]:
# Create model
x_train = np.expand_dims(scaled_training, axis=2)
y_train = scaled_training_labels

x_test = np.expand_dims(scaled_testing, axis=2)
y_test = scaled_testing_labels

x_test.shape

In [ ]:
# Define the model
model = Sequential()

model.add(Conv1D(32, 10, input_shape=(17867,1), padding='same', activation="relu"))
model.add(MaxPooling1D(200))
model.add(Flatten())
model.add(Dropout(0.20))

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.20))

model.add(Dense(4, activation="softmax"))

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print a summary of the model
model.summary()

In [ ]:
# Train the model
model.fit(
    x_train,
    y_train,
    epochs=30,
    validation_data=(x_test, y_test),
    shuffle=True
)

In [ ]:
test_error_rate = model.evaluate(x_test, y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

In [ ]:
# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure_conv_10.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model_weights_conv_10.h5")

# # Load the json file that contains the model's structure
# f = Path("data/ks2c_model_structure_conv_10.json")
# model_structure = f.read_text()
#
# # Recreate the Keras model object from the json data
# model = model_from_json(model_structure)
#
# # Re-load the model's trained weights
# model.load_weights("data/ks2c_model_weights_conv_10.h5")

In [ ]:
# Make a prediction with the neural network
X = np.expand_dims(rna_scaled_evaluation, axis=2)
rna_prediction = model.predict(X)
df_rna_pred = pd.DataFrame(data=rna_prediction, index=rna_data.index, columns=['Naive', 'IFN', 'LPS24', 'LPS2'])
df_rna_pred.style.background_gradient(cmap='viridis')